In [16]:
import json
import torch
from sentence_transformers import util
import os

In [17]:
big = "memoryalpha"
small = "stexpanded"

top_k = 10

mapping_file_big = "./data/triples_v2/" + big + "_mapping.json"
mapping_file_small = "./data/triples_v2/" + small + "_mapping.json"
exact_match_file = "./data/exact_match/" + big + "-" + small + ".json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

communitiy_embeddings_big_file = "./results/_community_embeddings/" + big + ".json"
communitiy_embeddings_small_file = "./results/_community_embeddings/" + small + ".json"

node_embeddings_big_file = "./results/embeddings_deepseek/" + big + "_summaries_v1_embeddings.json"
node_embeddings_small_file = "./results/embeddings_deepseek/" + small + "_summaries_v1_embeddings.json"

url_embeddings_big_file = "./results/url_embeddings_short/" + big + "_url_BAAI_bge-large-en-v1.5.json"
url_embeddings_small_file = "./results/url_embeddings_short/" + small + "_url_BAAI_bge-large-en-v1.5.json"

matched_communities_path = f"./results/_matched_nodes_communities_leiden/top_{top_k}_pairs/"

In [18]:
with open(exact_match_file) as emf, open(mapping_file_big) as mfb, open(mapping_file_small) as mfs:
    mapping_big = json.load(mfb)
    mapping_small = json.load(mfs)
    exact_match = json.load(emf)

In [19]:
small_communities = []
big_communities = []

with open(communities_big_file) as cbf, open(communities_small_file) as csf:
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)

In [20]:
with open(communitiy_embeddings_big_file) as cebf, open(communitiy_embeddings_small_file) as cesf:
    community_embeddings_big = json.load(cebf)
    community_embeddings_big = {k: list(v.values()) for k, v in community_embeddings_big.items()}
    community_embeddings_small = json.load(cesf)
    community_embeddings_small = {k: list(v.values()) for k, v in community_embeddings_small.items()}

In [21]:
with open(node_embeddings_big_file) as nebf, open(node_embeddings_small_file) as nesf:
    node_embeddings_big = json.load(nebf)
    node_embeddings_small = json.load(nesf)

In [22]:
for k, v in node_embeddings_big.items():
    print(k)
    print(v)
    break

0
[0.01786981336772442, -0.00037676034844480455, 0.008939134888350964, 0.041433751583099365, -0.02004345692694187, -0.0395379476249218, -0.04578113928437233, -0.022421427071094513, 0.009334806352853775, -0.01677742972970009, 0.0278811976313591, -0.06367318332195282, 0.012302359566092491, -0.02252109721302986, -0.03505859524011612, -0.05650084838271141, 0.029093217104673386, -0.002350793220102787, -0.046256572008132935, -0.030010078102350235, 0.04475285857915878, 0.01373687107115984, -0.049506574869155884, -0.05602056160569191, -0.025687135756015778, 0.005204776301980019, 0.05963103473186493, -0.016754120588302612, 0.048814766108989716, 0.027259523048996925, -0.02417297661304474, -0.04314543679356575, -0.01223958283662796, -0.03533006086945534, 0.030623909085989, 0.001090924721211195, 0.023816339671611786, -0.030671376734972, -0.008997197262942791, -0.06115114316344261, 0.006934798322618008, -0.01798476278781891, 0.042850982397794724, -0.017443854361772537, 0.0003036597336176783, 0.0350

In [23]:
with open(url_embeddings_big_file) as uebf, open(url_embeddings_small_file) as uesf:
    url_embeddings_big = json.load(uebf)
    url_embeddings_small = json.load(uesf)

In [24]:
merged_node_embeddings_big = {key: node_embeddings_big[key] if key in node_embeddings_big else url_embeddings_big[key] for key in set(node_embeddings_big) | set(url_embeddings_big)}
merged_node_embeddings_big = {str(k): merged_node_embeddings_big[str(k)] for k in sorted(map(int, merged_node_embeddings_big.keys()))}

merged_node_embeddings_small = {key: node_embeddings_small[key] if key in node_embeddings_small else url_embeddings_small[key] for key in set(node_embeddings_small) | set(url_embeddings_small)}
merged_node_embeddings_small = {str(k): merged_node_embeddings_small[str(k)] for k in sorted(map(int, merged_node_embeddings_small.keys()))}

In [25]:
big_torch_embeds = torch.Tensor(list(community_embeddings_big.values()))
small_torch_embeds = torch.Tensor(list(community_embeddings_small.values()))
big_torch_node_embeds = torch.Tensor(list(merged_node_embeddings_big.values()))
small_torch_node_embeds = torch.Tensor(list(merged_node_embeddings_small.values()))

pair_top_k = util.semantic_search(big_torch_node_embeds, small_torch_embeds, top_k=top_k)
reverse_pair_top_k = util.semantic_search(small_torch_node_embeds, big_torch_embeds, top_k=top_k)

In [26]:
pair_top_k[0]

[{'corpus_id': 293, 'score': 0.7599117159843445},
 {'corpus_id': 12, 'score': 0.7495840191841125},
 {'corpus_id': 81, 'score': 0.7413595914840698},
 {'corpus_id': 277, 'score': 0.7383219003677368},
 {'corpus_id': 368, 'score': 0.7358753681182861},
 {'corpus_id': 276, 'score': 0.7339476346969604},
 {'corpus_id': 1089, 'score': 0.7318533062934875},
 {'corpus_id': 114, 'score': 0.730922281742096},
 {'corpus_id': 292, 'score': 0.7299650311470032},
 {'corpus_id': 116, 'score': 0.7287887930870056}]

In [27]:
reverse_pair_top_k[0]

[{'corpus_id': 942, 'score': 0.7514848709106445},
 {'corpus_id': 1050, 'score': 0.7449561357498169},
 {'corpus_id': 1051, 'score': 0.7445398569107056},
 {'corpus_id': 935, 'score': 0.7410078048706055},
 {'corpus_id': 938, 'score': 0.739598274230957},
 {'corpus_id': 57, 'score': 0.7387492060661316},
 {'corpus_id': 1054, 'score': 0.7379314303398132},
 {'corpus_id': 1154, 'score': 0.7370851635932922},
 {'corpus_id': 2023, 'score': 0.7356512546539307},
 {'corpus_id': 923, 'score': 0.7350783348083496}]

In [28]:
forward_dict = dict()
backward_dict = dict()
g1_keys = list(community_embeddings_big.keys())
g2_keys = list(community_embeddings_small.keys())
g1_node_keys = list(merged_node_embeddings_big.keys())
g2_node_keys = list(merged_node_embeddings_small.keys())

for a, b in zip(g1_node_keys, pair_top_k):
    row_info = list()
    for element in b:
        row_info.append([g2_keys[element["corpus_id"]], element["score"]])
    forward_dict[str(a)] = row_info

for a, b in zip(g2_node_keys, reverse_pair_top_k):
    row_info = list()
    for element in b:
        row_info.append([g1_keys[element["corpus_id"]], element["score"]])
    backward_dict[str(a)] = row_info

In [29]:
with open(os.path.join(matched_communities_path, f"{big}-{small}_top_{str(top_k)}_pairs.json"), "w") as f:
    json.dump(forward_dict, f)

with open(os.path.join(matched_communities_path, f"{small}-{big}_top_{str(top_k)}_pairs.json"), "w") as f:
    json.dump(backward_dict, f)